# Visualization using WriteVTK
This notebook shows how to do mesh and data visualization using the ``WriteVTK.jl`` package. The package allows us to build a `.vtu` file containing mesh and data (located either on the nodes or the elements, scalar or vector). This data can then be visualized using an external program such as [ParaView](https://www.paraview.org/).

In [1]:
using gmsh

using WriteVTK

# Define the Mesh
First we retrieve the nodes (which are called _points_ in VTK terminology) and elements (which are called _cells_) from Gmsh. These are then used to create the VTK file structure with the right mesh information.

In [ ]:
# Nodes / points
#  these are stored in a single vector with the formatting [x1, y1, z1, x2, y2, z2, ...]
node_ids, node_coord, _ = gmsh.model.mesh.getNodes()
xnode = node_coord[1:3:end];
ynode = node_coord[2:3:end];

# Elements / cells
eType, eTag, eConn = gmsh.model.mesh.getElements(2);

elIds = 1:length(eTag[1]);
els = [zeros(Int, 3) for i in elIds];
for eid in elIds
    els[eid] = [eConn[1][3*(eid-1)+1], eConn[1][3*(eid-1)+2], eConn[1][3*(eid-1)+3]]
end

Define the points, which is a $2 \times N_{node}$ matrix of $(x,y)$ pairs. The cells is an array of length $N_{elem}$ where each element is a structure encoding the connectivity of the cell: `MeshCell(type, connectivity)`. We exclusively use triangle elements in this course. 

In [ ]:
# Define nodes (points) and elements (cells)
points = [xnode ynode]';
cells = [MeshCell(VTKCellTypes.VTK_TRIANGLE, el) for el in els];

# Create VTK file structure using nodes and elements
vtkfile = vtk_grid("path/to/file.vtu", points, cells);

We can then store point or cell data in the file as follows.

In [ ]:
vtkfile["point_data_name", VTKPointData()] = point_data;
vtkfile["cell_data_name", VTKCellData()]   = cell_data;

Finally, the file needs to be written, after which it can be opened in an external VTK viewer.

In [ ]:
# Save the file
outfiles = vtk_save(vtkfile);

# Example: Square with Hole

In [2]:
gmsh.initialize()
gmsh.open("geo_files/square_hole.msh")

Info    : Reading 'geo_files/square_hole.msh'...
Info    : 17 entities
Info    : 116 nodes
Info    : 240 elements
Info    : Done reading 'geo_files/square_hole.msh'


In [3]:
# Nodes / points
#  these are stored in a single vector with the formatting [x1, y1, z1, x2, y2, z2, ...]
node_ids, node_coord, _ = gmsh.model.mesh.getNodes()
xnode = node_coord[1:3:end];
ynode = node_coord[2:3:end];

# Elements / cells
eType, eTag, eConn = gmsh.model.mesh.getElements(2);

elIds = 1:length(eTag[1]);
els = [zeros(Int, 3) for i in elIds];
for eid in elIds
    els[eid] = [eConn[1][3*(eid-1)+1], eConn[1][3*(eid-1)+2], eConn[1][3*(eid-1)+3]]
end

In [4]:
# Define nodes (points) and elements (cells)
points = [xnode ynode]';
cells = [MeshCell(VTKCellTypes.VTK_TRIANGLE, el) for el in els];

# Create VTK file structure using nodes and elements
vtkfile = vtk_grid("figures/square_hole.vtu", points, cells);

# Save the file
outfiles = vtk_save(vtkfile);

![ParaView: Square with Hole](figures/square_hole_paraview.png)

# Example: Coaxial Cable

In [5]:
gmsh.initialize()
gmsh.open("geo_files/coaxial_cable.msh")

Info    : Reading 'geo_files/coaxial_cable.msh'...
Info    : 11 entities
Info    : 985 nodes
Info    : 1904 elements
Info    : Done reading 'geo_files/coaxial_cable.msh'


In [6]:
# Nodes / points
#  these are stored in a single vector with the formatting [x1, y1, z1, x2, y2, z2, ...]
node_ids, node_coord, _ = gmsh.model.mesh.getNodes()
xnode = node_coord[1:3:end];
ynode = node_coord[2:3:end];

# Elements / cells
eType, eTag, eConn = gmsh.model.mesh.getElements(2);

elIds = 1:length(eTag[1]);
els = [zeros(Int, 3) for i in elIds];
for eid in elIds
    els[eid] = [eConn[1][3*(eid-1)+1], eConn[1][3*(eid-1)+2], eConn[1][3*(eid-1)+3]]
end

In [9]:
#..5/12 Create groups of elements for the subdomains
#..for loop that creates a vector describing which physical group an element belongs to
ngroup1 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 1)
ngroup2 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 2)
e_group = zeros(1, length(els))
for (element_id, nodes) in enumerate(els)
    node1_id = nodes[1]; node2_id = nodes[2]; node3_id = nodes[3];
    G1 = sum(node1_id.== ngroup1[1])+sum(node2_id.== ngroup1[1])+sum(node3_id.== ngroup1[1]) # Conductor
    G2 = sum(node1_id.== ngroup2[1])+sum(node2_id.== ngroup2[1])+sum(node3_id.== ngroup2[1]) # Insulator
    if G1 == 3
        e_group[element_id] = 1;
    elseif G2 == 3
        e_group[element_id] = 2;
    end
end

In [10]:
# Define nodes (points) and elements (cells)
points = [xnode ynode]';
cells = [MeshCell(VTKCellTypes.VTK_TRIANGLE, el) for el in els];

# Create VTK file structure using nodes and elements
vtkfile = vtk_grid("figures/coaxial_cable.vtu", points, cells);
vtkfile["group", VTKCellData()] = e_group;

# Save the file
outfiles = vtk_save(vtkfile);

![ParaView: Coaxial Cable](figures/coaxial_cable_paraview.png)